In [1]:
#thining operator

In [2]:
import numpy as np
import cv2
np.set_printoptions(threshold=np.inf)

In [3]:
yokoi = np.loadtxt('matrix.txt')
img = cv2.imread('iteration_12_img.bmp',cv2.IMREAD_GRAYSCALE)

In [4]:
#pair relationship operator

#先把yokoi做padding:
paddingyokoi = np.zeros([66,66])
for row in range(64):
    for col in range(64):
        paddingyokoi[row+1][col+1] = yokoi[row][col]
        
#區分原本是零與經過h(a)後成為零的值，因此先把原本值為零改成10
for row in range(66):
    for col in range(66):
        if paddingyokoi[row][col] == 0 :
            paddingyokoi[row][col] = 10

#新開一張
markedimg = np.zeros([66,66])
for row in range(66):
    for col in range(66):
        markedimg[row][col] = paddingyokoi[row][col]

#h()
for row in range(66):
    for col in range(66):
        if paddingyokoi[row][col] == 1:
            markedimg[row][col] = 1
        elif paddingyokoi[row][col] == 10:
            markedimg [row][col] = 10
        else:
            markedimg[row][col] = 0
#print('markedimg=', markedimg)

#開設一張markedimg2存markedimg的data, 把markedimg中10的位置改回0，以便等等判斷
markedimg2 = np.zeros([66,66])
for row in range(66):
    for col in range(66):
        markedimg2[row][col] = markedimg[row][col]
        if markedimg[row][col] == 10:
            markedimg[row][col] = 0

#print('markedimg2=',markedimg2)
#print('markedimg=',markedimg)

In [5]:
#pair_output
sum = 0
finalmarkedimg = np.zeros([66,66]) #一樣有padding
for row in range(64):
    for col in range(64):
        if markedimg2[row+1][col+1] == 0 or markedimg2[row+1][col+1] == 1: #10的位置，也就是原本=0的位置不用判斷。
            sum = markedimg[row+1][col+2] + markedimg[row][col+1] + markedimg[row+1][col] + markedimg[row+2][col+1]
            
            if sum < 1 or markedimg2[row+1][col+1] != 1:
                finalmarkedimg[row+1][col+1] = 9 # 設q = 9
                sum = 0
            elif sum >= 1 and markedimg2[row+1][col+1] == 1:
                finalmarkedimg[row+1][col+1] = 6 # 設p = 6
                sum = 0
            else:
                print('error')
                sum = 0
#print(markedimg)
#print(markedimg2)
#print(finalmarkedimg)
                
                

In [6]:
#print(np.shape(img))
#print(finalmarkedimg)
#padding img
paddingimg = np.zeros([66,66])
for row in range(64):
    for col in range(64):
        paddingimg[row+1][col+1] = img[row][col]
#so 現在有原圖和標記q,p後的圖
#針對標記p(6)的位置判斷 connected shrink operator(以原圖數值判斷(0,255)):

#開a1,a2,a3,a4空間，都先填5，這樣等等填0,1，5 之後加總才知道位置在哪，不然會搞混0
a1 = np.zeros([64,64])
a2 = np.zeros([64,64])
a3 = np.zeros([64,64])
a4 = np.zeros([64,64])
for row in range(64):
    for col in range(64):
        a1[row][col]=5
        a2[row][col]=5
        a3[row][col]=5
        a4[row][col]=5
        
        

#def h(b,c,d,e):
def h(b,c,d,e):
    if b == c:
        if b!=d or b!=e:
            return 1
        else:
            return 0
    else:
        return 0

for row in range(64):
    for col in range(64):
        if finalmarkedimg[row+1][col+1] == 6:
            #a1 = h(x0,x1,x6,x2)
            #a2 = h(x0,x2,x7,x3)
            #a3 = h(x0,x3,x8,x4)
            #a4 = h(x0,x4,x5,x1)
            #h(b,c,d,e) = 1 if b=c and (b!=d or b!=e)
            
            
            a1[row][col] = h(paddingimg[row+1][col+1],paddingimg[row+1][col+2],paddingimg[row][col+2],paddingimg[row][+1])
            #print('b=',paddingimg[row+1][col+1],'c=',paddingimg[row+1][col+2],'d=',paddingimg[row][col+2],'e=',paddingimg[row][+1])
            #print('a1=',a1[row][col])
            a2[row][col] = h(paddingimg[row+1][col+1],paddingimg[row][col+1],paddingimg[row][col],paddingimg[row+1][col])
            #print('b=',paddingimg[row+1][col+1],'c=',paddingimg[row][col+1],'d=',paddingimg[row][col],'e=',paddingimg[row+1][col])
            #print('a2=',a2[row][col])
            a3[row][col] = h(paddingimg[row+1][col+1],paddingimg[row+1][col],paddingimg[row+2][col],paddingimg[row+2][col+1])
            #print('b=',paddingimg[row+1][col+1],'c=',paddingimg[row+1][col],'d=',paddingimg[row+2][col],'e=',paddingimg[row+2][col+1])
            #print('a3=',a3[row][col])
            a4[row][col] = h(paddingimg[row+1][col+1],paddingimg[row+2][col+1],paddingimg[row+2][col+2],paddingimg[row+1][col+2])
            #print('b=',paddingimg[row+1][col+1],'c=',paddingimg[row+2][col+1],'d=',paddingimg[row+2][col+2],'e=',paddingimg[row+1][col+2])
            #print('a4=',a4[row][col])
            
            

In [7]:
#print(np.shape(finalmarkedimg))
#print(np.shape(paddingimg))

In [8]:
#print('a1=',a1)
#print('a2=',a2)
#print('a3=',a3)
#print('a4=',a4)

In [9]:
#加總各個位置
sum = np.zeros([64,64])
for row in range(64):
    for col in range(64):
        sum[row][col] = a1[row][col] + a2[row][col] + a3[row][col] + a4[row][col]
#print(sum)

In [10]:
#把sum剛好為1的值的位置在原圖上刪除，其他保留

for row in range(64):
    for col in range(64):
        if sum[row][col]==1:
            img[row][col] = 0


In [11]:
cv2.imshow('img',img)
cv2.waitKey()
cv2.imwrite('iteration_13_img.bmp',img)

True

In [12]:
#改輸入與輸出的image即可，疊代多次